In [1]:
from pathlib import Path
from utils import load_dataset, home

raw = home / 'data' / 'raw'
raw = load_dataset(raw)

DJIA_table_train (1584, 8)
RedditNews_train (58908, 2)
Combined_News_DJIA_train (1584, 27)


## DJIA

Stock market infomation

In [2]:
dija = raw['DJIA_table_train'].copy()
dija.head()

,Open,High,Low,Close,Volume,Adj Close,label
Date,,,,,,,
2008-08-08,11432.089844,NaN,11388.040039,11734.320312,212830000.0,11734.320312,0.0
2008-08-11,11729.669922,11867.110352,11675.530273,11782.349609,NaN,11782.349609,0.0
2008-08-12,11781.700195,11782.349609,11601.519531,11642.469727,173590000.0,11642.469727,1.0
2008-08-13,11632.809570,11633.780273,11453.339844,11532.959961,182550000.0,11532.959961,1.0
2008-08-14,11532.070312,11718.280273,11450.889648,11615.929688,159790000.0,11615.929688,NaN


In [3]:
def inspect_df(df):
    print('shape {}'.format(df.shape))
    print(' ')
    
    print('nulls')
    print('---')
    for col in df.columns:
        print(col, sum(df.loc[:, col].isnull()))
        
inspect_df(dija)

shape (1584, 7)
 
nulls
---
Open 74
High 79
Low 65
Close 71
Volume 92
Adj Close 61
label 81


Want to check the label column - to do this we need to compare with the value before:

In [4]:
def generate_labels(data):
    data = data.copy()
    #  small bug with the first day - should be nan
    data.loc[:, 'close-day-before'] = data.loc[:, 'Close'].shift(1)
    data.loc[:, 'mask'] = data['Close'] >= data['close-day-before']
    data.loc[:, 'label'] = data['mask'].astype('int64')
    return data
    
def test_generate_labels():
    data = pd.DataFrame({
        'Close': [0, 1, 0, 1, 2]
    })
    
    expected_labels = [0, 1, 0, 1, 1]
    labels = generate_labels(data)
    assert all(expected_labels == labels.loc[:, 'label'].values)
    return labels
    
d = test_generate_labels()
expected_labels = generate_labels(dija)
sum(expected_labels['label'] == dija['label']) - dija.shape[0]
#  all the labels are incorrect!

-1504

In [5]:
dija.loc[:, 'original-label'] = dija.loc[:, 'label']
dija.loc[:, 'label'] = expected_labels.loc[:, 'label']
sum(expected_labels['label'] == dija['label']) - dija.shape[0]

0

In [6]:
dija

,Open,High,Low,Close,Volume,Adj Close,label,original-label
Date,,,,,,,,
2008-08-08,11432.089844,NaN,11388.040039,11734.320312,212830000.0,11734.320312,0,0.0
2008-08-11,11729.669922,11867.110352,11675.530273,11782.349609,NaN,11782.349609,1,0.0
2008-08-12,11781.700195,11782.349609,11601.519531,11642.469727,173590000.0,11642.469727,0,1.0
2008-08-13,11632.809570,11633.780273,11453.339844,11532.959961,182550000.0,11532.959961,0,1.0
2008-08-14,11532.070312,11718.280273,11450.889648,11615.929688,159790000.0,11615.929688,1,NaN
...,...,...,...,...,...,...,...,...
2014-11-14,17653.109375,NaN,17613.199219,17634.740234,72850000.0,17634.740234,0,1.0
2014-11-17,17631.849609,17675.070312,17606.810547,17647.750000,70330000.0,17647.750000,1,0.0
2014-11-18,17643.089844,17735.710938,17642.029297,17687.820312,87420000.0,17687.820312,1,0.0


## Combined

In [7]:
comb = raw['Combined_News_DJIA_train'].copy()
comb.head(2)

,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
Date,,,,,,,,,,,,,,,,,,,,,,,,,,
2008-08-08,0.0,"b""Georgia 'downs two Russian warplanes' as cou...",NaN,b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",b'Georgian troops retreat from S. Osettain cap...,b'Did the U.S. Prep Georgia for War with Russia?',NaN,b'Announcing:Class Action Lawsuit on Behalf of...,"b""So---Russia and Georgia are at war and the N...","b""China tells Bush to stay out of other countr...",b'Did World War III start today?',b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
2008-08-11,1.0,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,b'Welcome To World War IV! Now In High Definit...,"b""Georgia's move, a mistake of monumental prop...",b'Russia presses deeper into Georgia; U.S. say...,b'Abhinav Bindra wins first ever Individual Ol...,b' U.S. ship heads for Arctic to define territ...,b'Drivers in a Jerusalem taxi station threaten...,b'The French Team is Stunned by Phelps and the...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',NaN


In [8]:
#comb = comb.set_index('Date', drop=True)
ds = pd.concat([comb, dija], axis=1)
ds.head(2)

,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,Open,High,Low,Close,Volume,Adj Close,label,original-label
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2008-08-08,0.0,"b""Georgia 'downs two Russian warplanes' as cou...",NaN,b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",b'Georgian troops retreat from S. Osettain cap...,b'Did the U.S. Prep Georgia for War with Russia?',NaN,b'Announcing:Class Action Lawsuit on Behalf of...,"b""So---Russia and Georgia are at war and the N...","b""China tells Bush to stay out of other countr...",b'Did World War III start today?',b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge""",11432.089844,NaN,11388.040039,11734.320312,212830000.0,11734.320312,0,0.0
2008-08-11,1.0,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,b'Welcome To World War IV! Now In High Definit...,"b""Georgia's move, a mistake of monumental prop...",b'Russia presses deeper into Georgia; U.S. say...,b'Abhinav Bindra wins first ever Individual Ol...,b' U.S. ship heads for Arctic to define territ...,b'Drivers in a Jerusalem taxi station threaten...,b'The French Team is Stunned by Phelps and the...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',NaN,11729.669922,11867.110352,11675.530273,11782.349609,NaN,11782.349609,1,0.0


In [9]:
eq_mask = ds.loc[:, 'Label'] == ds.loc[:, 'label']
sum(eq_mask)

1425

Below we can see that the disagreements between the `Label` col (from combine) and `label` (from DIJA) occur when we have nulls in either `Label` or in `Close`:

In [10]:
ds.loc[~eq_mask, ['Label', 'label', 'Close', 'original-label']].head()

,Label,label,Close,original-label
Date,,,,
2008-09-03,NaN,1,11532.879883,0.0
2008-09-26,1.0,0,NaN,0.0
2008-09-30,1.0,0,10850.660156,0.0
2008-10-03,NaN,0,10325.379883,1.0
2008-10-20,NaN,1,9265.429688,0.0


## Conclusions

We can trust the combined labels, but not the DIJA labels

## Cleaning draft

Filling in nulls in the label, using combined + DIJA:

In [11]:
dija = raw['DJIA_table_train'].copy()
expected_labels = generate_labels(dija)
dija_labels = expected_labels.loc[:, 'label'].to_frame()
dija_labels.to_csv(home / 'data' / 'interim' / 'dija-labels.csv')

In [12]:
comb = raw['Combined_News_DJIA_train'].copy()

news_cols = [c for c in comb.columns if 'Top' in c]

for name in news_cols:
    col = comb.loc[:, name]
    col = col.fillna(' ')
    col = col.apply(lambda x: x.strip('b'))
    col = col.apply(lambda x: x.strip('"'))
    col = col.apply(lambda x: x.strip("'"))
    comb.loc[:, name] = col

In [13]:
comb.loc[:, 'final-label'] = comb.loc[:, 'Label'].fillna(dija_labels.loc[:, 'label'])

In [14]:
sum(comb.loc[:, 'final-label'].isnull())

0

In [15]:
comb = comb.iloc[:, 1:].fillna(" ")
print(comb.head(1))

inspect_df(comb)

comb.to_csv(home / 'data' / 'interim' / 'combined.csv')

                                                         Top1 Top2                                               Top3                                               Top4                                               Top5                                               Top6                                               Top7                                               Top8                                               Top9                                           Top10 Top11                                              Top12                                              Top13                                              Top14                           Top15                                              Top16                             Top17                                              Top18                                              Top19                                              Top20                                              Top21  \
Date                                         